# PINN #1

## Import Relevant Packages

In [1]:
import pandas as pd
import sciann as sn
import numpy as np
from sklearn.preprocessing import MinMaxScaler

---------------------- SCIANN 0.6.1.1 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 

 Need support or would like to contribute, please join sciann`s slack group: 
 +  "https://join.slack.com/t/sciann/shared_invite/zt-ne1f5jlx-k_dY8RGo3ZreDXwz0f~CeA" 
 


## Load Data from Data Folder

In [2]:
import glob

df = pd.concat(map(pd.read_csv, glob.glob('data/*.csv')))
df

,Qf,Time,X,Y,Alpha,Temp(degC)
0,520,0.0,0.5,2.00,2.671200e-07,20.000000
1,520,0.0,1.0,2.00,2.671200e-07,20.000000
2,520,0.0,1.5,2.00,2.671200e-07,20.000000
3,520,0.0,2.0,2.00,2.671200e-07,20.000000
4,520,0.0,2.5,2.00,2.671200e-07,20.000000
...,...,...,...,...,...,...
3095,600,15.0,50.0,1.92,9.769047e-01,160.016833
3096,600,15.0,50.0,1.94,9.769034e-01,160.013898
3097,600,15.0,50.0,1.96,9.769021e-01,160.011001
3098,600,15.0,50.0,1.98,9.769008e-01,160.008143


In [3]:
# Shuffle the data so it's easy for the model to learn
df = df.sample(frac=1).reset_index(drop=True)
df

,Qf,Time,X,Y,Alpha,Temp(degC)
0,700,2.0,14.0,2.00,0.073831,48.049250
1,580,10.5,28.5,1.00,0.892433,123.835828
2,520,0.5,31.0,1.00,0.006093,25.434961
3,560,3.5,23.0,0.00,0.174738,58.343743
4,520,13.0,2.5,0.00,0.933439,129.560993
...,...,...,...,...,...,...
170495,640,11.0,5.0,0.00,0.939336,138.482800
170496,580,5.0,7.5,1.00,0.359191,74.351766
170497,540,10.5,50.0,0.88,0.856952,115.612310
170498,500,9.5,47.5,0.00,0.753843,101.895268


In [4]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
df[df.isna().any(axis=1)]

,Qf,Time,X,Y,Alpha,Temp(degC)


## Initialize MinMaxScaler

In [5]:
scaler = MinMaxScaler(feature_range = (0,1))

 ## Convert columns into NumPy Arrays and Scale Them

In [6]:
Q_np = df['Qf']
t_np = df['Time']
x_np = df['X']
y_np = df['Y']
alpha_np = df['Alpha']
temp_np = df['Temp(degC)']

In [7]:
Q_np = (Q_np.to_numpy())
t_np = (t_np.to_numpy())
x_np = (x_np.to_numpy())
y_np = (y_np.to_numpy())
alpha_np = (alpha_np.to_numpy())
temp_np = (temp_np.to_numpy())

## Define Inputs and Targets for PINN

In [8]:
# Define the necessary inputs and parameter using Variable and Parameter
t = sn.Variable('t')
x = sn.Variable('x')
y = sn.Variable('y')
Q = sn.Variable('Q')

# Define Neural Network Approximations
T = sn.Functional('T', [t,x,y,Q], 8*[20], 'tanh')
a = sn.Functional('a', [t,x,y,Q], 8*[20], 'tanh')

## Define Constants and PDEs

In [9]:
# Constants

H = 352.1 #1
A = 2e5 #2
E_a = 51100 #3 
n = 1.927 #4
p = 980 #5
c_p = 1440 #6
k = 1.8 #7
R = 8.314 #8
T_o = 20 #9
a_o = 0 #10
h = 1.8 #11

# PDEs
from sciann.utils.math import diff, sign, exp, pow

L1 = (k/(p*c_p))*diff(T, x, order=2) + (k/(p*c_p))*diff(T, y, order=2) + (H/c_p)*diff(a, t) - diff(T, t)
L2 = A*exp((-E_a/(R*T))*pow(1-a, n))*(1 + 0.36*a) - diff(a, t)

## Define Scaling Parameters

In [10]:
from sciann import Parameter

# Temperature Parameters
lambda1 = Parameter(val=0.0, min_max = [0,1e-6], inputs=[t,x,y], name="lambda1")
lambda2 = Parameter(val=0.0, min_max = [0,1e-6], inputs=[t,x,y], name="lambda2")
lambda3 = Parameter(val=0.0, min_max = [0,1e-6], inputs=[t,x,y], name="lambda3")
lambda4 = Parameter(val=0.0, min_max = [0,1e-6], inputs=[t,x,y], name="lambda4")
lambda5 = Parameter(val=0.0, min_max = [0,1e-6], inputs=[t,x,y], name="lambda5")

# Alpha Parameters
lambda6 = Parameter(val=0.0, min_max = [0,1e-6], inputs=[t,x,y], name="lambda6")

## Define Initial & Boundary Conditions

In [11]:
# t: [0,15]
# x: [0,50]
# y: [0,2]
# Q: [500,700]

tmin = 0
Xmin = 0
Xmax = 50
Ymin = 0
Ymax = 2

# Boundary Conditions
L3 = (k*diff(T, x))*(1-sign(x - Xmin)) + (h*T)*(1-sign(x - Xmin)) - (h*T_o)*(1-sign(x - Xmin))

L4 = (k*diff(T, x))*(1+sign(x - Xmax)) + (h*T)*(1+sign(x - Xmax)) - (h*T_o)*(1+sign(x - Xmax))

L5 = (k*diff(T, y))*(1-sign(y - Ymin)) + (h*T)*(1-sign(y - Ymin)) - (h*T_o)*(1-sign(y - Ymin))

L6 = (k*diff(T,y) - Q)*(1+sign(y - Ymax))

# Initial Conditions
L7 = (1-sign(t - tmin))*(T-20)

L8 = (1-sign(t - tmin))*(a)

# Combined
L_T = L1 + lambda1*L3 + lambda2*L4 + lambda3*L5 + lambda4*L6 + lambda5*L7

L_a = L2 + lambda6*L8

# Data
L_T_DATA = T

L_a_DATA = a

## Build the Model

In [12]:
m = sn.SciModel([t,x,y,Q], [L_T, L_a, L_T_DATA, L_a_DATA])

In [13]:
h = m.train(
    [t_np, x_np, y_np, Q_np], 
    ['zeros','zeros', temp_np, alpha_np], 
    epochs=20000,
    batch_size=128,
    verbose = 0,
    shuffle=False)


Total samples: 170500 
Batch size: 128 
Total batches: 1333 


Epoch 03881: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 05881: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 07881: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 09881: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 11881: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.

Epoch 13881: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.

Epoch 15881: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.

Epoch 17881: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.

Epoch 19881: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-06.


## Getting Test Data

In [14]:
test_df = pd.concat(map(pd.read_csv, glob.glob('test_data/*.csv')))
test_df.head()

,Qf,Time,X,Y,Alpha,Temp(degC)
0,440,0.0,0.5,2.0,2.671200e-07,20.0
1,440,0.0,1.0,2.0,2.671200e-07,20.0
2,440,0.0,1.5,2.0,2.671200e-07,20.0
3,440,0.0,2.0,2.0,2.671200e-07,20.0
4,440,0.0,2.5,2.0,2.671200e-07,20.0


In [15]:
test_df = test_df.sample(frac=1).reset_index(drop=True)
test_df.head()

,Qf,Time,X,Y,Alpha,Temp(degC)
0,400,13.5,18.0,2.00,0.856245,107.146962
1,420,5.5,0.0,0.46,0.272735,62.637489
2,460,2.5,50.0,1.18,0.073313,42.568732
3,480,7.5,6.5,0.00,0.544661,84.485582
4,480,7.0,0.0,1.48,0.485032,80.125314


In [16]:
test_df[test_df.isna().any(axis=1)]

,Qf,Time,X,Y,Alpha,Temp(degC)


## Getting Predictions

In [17]:
Q_test_np = test_df['Qf']
t_test_np = test_df['Time']
x_test_np = test_df['X']
y_test_np = test_df['Y']
alpha_test_np = test_df['Alpha']
temp_test_np = test_df['Temp(degC)']

Q_test_np = (Q_test_np.to_numpy().reshape(-1,1))
t_test_np = (t_test_np.to_numpy().reshape(-1,1))
x_test_np = (x_test_np.to_numpy().reshape(-1,1))
y_test_np = (y_test_np.to_numpy().reshape(-1,1))
alpha_test_np = (alpha_test_np.to_numpy().reshape(-1,1))
temp_test_np = (temp_test_np.to_numpy().reshape(-1,1))

In [18]:
T_pred = T.eval([t_test_np, x_test_np, y_test_np, Q_test_np])
a_pred = a.eval([t_test_np, x_test_np, y_test_np, Q_test_np])

## Results

In [19]:
from sklearn.metrics import mean_squared_error as mse
import math
print('TEMPERATURE: ')
print('Test:  ', round(math.sqrt(mse(temp_test_np, T_pred)), 3))
print(' ')
print('ALPHA: ')
print('Test:  ', round(math.sqrt(mse(alpha_test_np, a_pred)), 3))

TEMPERATURE: 
Test:   37.155
 
ALPHA: 
Test:   0.333
